In [ ]:
import pandas as pd
import os

In [ ]:
dir_saida = "/home/gabrielacastilho/Jo/diamond_comparacao_200_CDS"

In [1]:
arquivo_delta = "/home/gabrielacastilho/Jo/idxstats/CDS_delta_PD_vs_Control.tsv"
arquivo_db6 = "/home/gabrielacastilho/Jo/diamond_6_without_extra_aligned/6_matches_concat.tsv"

saida = "/home/gabrielacastilho/Jo/diamond_6_without_extra_aligned/DB6_top200_delta.tsv"

In [ ]:
df_db6 = pd.read_csv(arquivo_db6, sep="\t")
cds_db6 = set(df_db6["CDS"].unique())
df_delta = pd.read_csv(arquivo_delta, sep="\t")
df_delta.columns = df_delta.columns.str.strip()

In [ ]:
df_delta_db6 = df_delta[df_delta["CDS"].isin(cds_db6)]


In [ ]:
df_delta_db6 = df_delta_db6.drop_duplicates(subset=["CDS"])

In [ ]:
print(f"CDS únicos com delta e presentes na DB6: {df_delta_db6.shape[0]}")


In [ ]:
df_delta_db6["abs_delta"] = df_delta_db6["delta_PD_vs_Control"].abs()

df_top200 = (
    df_delta_db6
    .sort_values("abs_delta", ascending=False)
    .head(200)
)

In [ ]:
cds_top200 = set(df_top200["CDS"])

print(f"CDS selecionados (top 200): {len(cds_top200)}")

In [ ]:
# Extrair DB6 (1 linha por CDS)
df_db6_unique = df_db6.drop_duplicates(subset=["CDS"])
df_db6_top200 = df_db6_unique[df_db6_unique["CDS"].isin(cds_top200)]

# Anexar delta (sem duplicar)
df_db6_top200 = df_db6_top200.merge(
    df_top200[["CDS", "delta_PD_vs_Control"]],
    on="CDS",
    how="left"
) 
# =========================
# Salvar
# =========================
df_db6_top200.to_csv(saida, sep="\t", index=False)

print(f"\nArquivo final salvo em:\n{saida}")

In [ ]:
df_final = pd.read_csv(saida, sep='\t')

df_final = df_final.sort_values(by='bitscore', ascending=False)
df_final.to_csv(saida, sep='\t', index=False)

In [ ]:
goals_cds= set(df_db6_top200['CDS'])


In [ ]:
resumo = []

for nome_base, caminho in bases.items():
    print(f"Processando {nome_base}...")

    df = pd.read_csv(caminho, sep="\t")

    # Filtra apenas CDS de interesse
    df_filtrado = df[df["CDS"].isin(goals_cds)]

    # Remove possíveis duplicatas CDS–Amostra (segurança extra)
    df_filtrado = df_filtrado.drop_duplicates(subset=["CDS", "Amostra"])

    # Salva arquivo
    arquivo_saida = os.path.join(
        dir_saida,
        f"200_CDS_DB6_vs_{nome_base}.tsv"
    )

    df_filtrado.to_csv(arquivo_saida, sep="\t", index=False)

    n_cds = df_filtrado["CDS"].nunique()

    resumo.append({
        "Base": nome_base,
        "CDS_anotados": n_cds,
        "Total_200": len(cds_interesse)
    })

    print(f"  → {n_cds} / {len(cds_interesse)} CDS anotados\n")

In [ ]:
df_resumo = pd.DataFrame(resumo)
df_resumo = df_resumo.sort_values("CDS_anotados", ascending=False)

resumo_saida = os.path.join(dir_saida, "resumo_cobertura_bases.tsv")
df_resumo.to_csv(resumo_saida, sep="\t", index=False)

print("Resumo final:")
print(df_resumo)
print(f"\nArquivos salvos em: {dir_saida}")